In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import requests


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


data/accidents-in-france-from-2005-to-2016/places.csv
data/accidents-in-france-from-2005-to-2016/holidays.csv
data/accidents-in-france-from-2005-to-2016/vehicles.csv
data/accidents-in-france-from-2005-to-2016/caracteristics.csv
data/accidents-in-france-from-2005-to-2016/users.csv


In [2]:
caracteristics_path = 'data/accidents-in-france-from-2005-to-2016/caracteristics.csv'
caracteristics = pd.read_csv(caracteristics_path, encoding = 'latin-1')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def invalide_rows(caracteristics):
    
    
    adr_na = caracteristics['adr'].isna()
    
    lat_na = caracteristics['lat'].isna() | caracteristics['lat'].isnull()
    long_na = caracteristics['long'].isna() | caracteristics['long'].isnull()
    
    
    lat_zero = caracteristics['lat'] == 0
    long_zero = (caracteristics['long'] == 0) | (caracteristics['long'] == '0')
    
    invalid_adr =  adr_na & (lat_zero | long_zero | lat_na | long_na)
    
    return invalid_adr

In [4]:
invalid_rows = invalide_rows(caracteristics)
valid_rows = ~invalid_rows

In [5]:
valide_caracteristics = caracteristics[valid_rows]
# garder tous ce qui sont dans la metropole
in_metropole = (valide_caracteristics['gps'] == 'M') | valide_caracteristics['gps'].isna() & (valide_caracteristics['dep'] < 951)
valide_caracteristics = valide_caracteristics[in_metropole]

In [6]:
def adr_could_be_filled(valide_caracteristics):
    
    adr_not_na = valide_caracteristics['adr'].notna()
    
    lat_na = valide_caracteristics['lat'].isna() | valide_caracteristics['lat'].isnull()
    long_na = valide_caracteristics['long'].isna() | valide_caracteristics['long'].isnull()
    
    code_com_dep_na  = valide_caracteristics['com'].isna() | valide_caracteristics['com'].isnull() 
    code_com_dep_na |= valide_caracteristics['dep'].isna() | valide_caracteristics['dep'].isnull()

    
    lat_zero = valide_caracteristics['lat'] == 0
    long_zero = (valide_caracteristics['long'] == 0) | (valide_caracteristics['long'] == '0')
    
    return adr_not_na & (~code_com_dep_na) & (lat_na | long_na | lat_zero | long_zero)
    

In [7]:
valide_adr = adr_could_be_filled(valide_caracteristics)

valide_adr_caracteristics = valide_caracteristics[valide_adr]

In [8]:
def formate_com(com):
    com = str(int(com))
    extra_zero = '0' * (3 - len(com))
    return extra_zero + com 


In [9]:
com_new = valide_adr_caracteristics['com'].map(lambda x: formate_com(x))
dep_new = valide_adr_caracteristics['dep'].map(lambda x: str(x)[:2])

valide_adr_caracteristics['dep_com'] = dep_new + com_new

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [10]:
import time
import requests
import threading
import numpy as np
import concurrent.futures

In [11]:
lock = threading.Lock()

def get_columns():
    return [
        ('city', ''), ('name', ''), ('score', float(0.0)), 
        ('postcode', -1), ('longitude', float(0.0)), ('latitude', float(0.0))
    ]

def add_columns(df):
    for col in get_columns():
        df[col[0]] = col[1]
    return df


start = time.time()
valide_adr_carac = valide_adr_caracteristics #.iloc[:1000]
valide_adr_carac = add_columns(valide_adr_carac)

url = 'http://api-adresse.data.gouv.fr/search/?q={}'
adr_dep_com = (valide_adr_carac['adr'] + ' ') + valide_adr_carac['dep_com']
valide_adr_carac['urls'] = adr_dep_com.map(url.format)


def lat_long_from_adr(index_carac):

    global lock
    i, carac = index_carac # valide_adr_caracteristics.iloc[i]
        
    response = requests.get(carac['urls'])
    resp_json = response.json()
    if len(resp_json['features']) < 1: return
    feature = resp_json['features'][0]
    
    global valide_adr_carac
    with lock:
        valide_adr_carac.loc[i, 'city'] = feature['properties']['city']
        valide_adr_carac.loc[i, 'name'] = feature['properties']['name']
        valide_adr_carac.loc[i, 'score'] = round(feature['properties']['score'], 3)
        valide_adr_carac.loc[i, 'postcode'] = feature['properties']['postcode']
        valide_adr_carac.loc[i, 'longitude'] = feature['geometry']['coordinates'][0]
        valide_adr_carac.loc[i, 'latitude'] = feature['geometry']['coordinates'][1]
    
    if i %1000 == 0:
        print("step " , i)
        
    
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(lat_long_from_adr, valide_adr_carac.iterrows())


print('end', time.time() - start, 's')
cols = np.array(get_columns())[:, 0]
print(cols)
valide_adr_carac.loc[:, cols].to_csv('adr_complet.csv', index_label='Index', index=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/chupin/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

step  0
step  6000
step  11000
step  38000
step  39000
step  40000
step  41000
step  42000
step  46000
step  47000
step  49000
step  50000
step  51000
step  65000
step  82000
step  85000
step  88000
step  89000
step  96000
step  97000
step  98000
step  99000
step  100000
step  101000
step  102000
step  103000
step  106000
step  107000
step  108000
step  109000
step  110000
step  111000
step  112000
step  113000
step  114000
step  115000
step  116000
step  125000
step  126000
step  128000
step  131000
step  136000
step  140000
step  141000
step  142000
step  143000
step  145000
step  149000
step  152000
step  154000
step  157000
step  158000
step  159000
step  160000
step  161000
step  166000
step  167000
step  168000
step  170000
step  171000
step  172000
step  173000
step  174000
step  175000
step  176000
step  179000
step  188000
step  189000
step  193000
step  199000
step  201000
step  202000
step  204000
step  205000
step  208000
step  211000
step  212000
step  215000
step  216000


KeyboardInterrupt: 

In [ ]:
invalide = valide_adr_carac[valide_adr_carac['postcode'] == -1]['postcode'].count() 
total = valide_adr_carac['postcode'].count()

print(invalide)
print(total)
'loss {} %'.format(invalide / total)

In [ ]:
#unsent_requests = (grequests.get(u, session=requests.Session(), stream=False) for u in urls)
#valide_adr_caracteristics = add_columns(valide_adr_caracteristics)
#responses = grequests.map(unsent_requests, size=10)
#for i, response in enumerate(responses):
#    lat_long_from_adr(response, valide_adr_caracteristics, i)